<a href="https://colab.research.google.com/github/cedro3/GFPGAN/blob/master/GFPGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title #**setup**

# --- GFPGAN setup ---
# Clone GFPGAN
%cd /content
!rm -rf GFPGAN
!git clone https://github.com/cedro3/GFPGAN.git
%cd GFPGAN

# install library
!pip install basicsr==1.3.5
!pip install facexlib
!pip install -r requirements.txt
!python setup.py develop
!pip install realesrgan  

# Download the pre-trained model
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P experiments/pretrained_models

# add function
from function import display_pic, reset_folder, comp_display


# --- DeOldify setup ---
# Clone DeOldify
%cd /content
!rm -rf DeOldify
!git clone https://github.com/jantic/DeOldify.git
%cd DeOldify

#NOTE:  This must be the first call in order to work properly!
from deoldify import device
from deoldify.device_id import DeviceId

#choices:  CPU, GPU0...GPU7
device.set(device=DeviceId.GPU0)
import torch
if not torch.cuda.is_available():
    print('GPU not available.')

# install library
! pip install -r colab_requirements.txt

# import library
import fastai
from deoldify.visualize import *
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*?Your .*? set is empty.*?")

# download models
! mkdir 'models'
! wget https://data.deepai.org/deoldify/ColorizeArtistic_gen.pth -O ./models/ColorizeArtistic_gen.pth

# download watermark
! wget https://media.githubusercontent.com/media/jantic/DeOldify/master/resource_images/watermark.png -O ./resource_images/watermark.png

# load model
colorizer = get_image_colorizer(artistic=True)


In [ ]:
#@title #**display sample picture**
display_pic('/content/GFPGAN/images')

In [ ]:
#@title #**GFPGAN**
%cd /content/GFPGAN
!rm -rf results
picture = '09.jpg'#@param {type:"string"}
input_path = 'images/'+ picture

!python inference_gfpgan.py -i $input_path -o results -v 1.3 -s 2 --bg_upsampler realesrgan

# display befor & after
from IPython.display import clear_output
clear_output()
img_input = cv2.imread('images/'+picture)
img_output = cv2.imread('results/restored_imgs/'+picture)
comp_display(img_input, img_output)

In [ ]:
#@title #**DeOldify**

# copy image
%cd /content/DeOldify
import shutil

pic = '/content/GFPGAN/results/restored_imgs/'+picture
dst_pic = '/content/DeOldify/test_images/'+picture
shutil.copy(pic, dst_pic)


# DeOldify
file_path = 'test_images/'+picture
render_factor = 35 

colorizer.plot_transformed_image(file_path,\
                                  render_factor= render_factor,\
                                  watermarked = False,\
                                  display_render_factor=False,\
                                  compare=True,\
                                  figsize=(15,15))

In [ ]:
#@title #**final comparison image**

# concatenate pre-conversion and post-conversion images
reset_folder('compare_images')
mono = cv2.imread('/content/GFPGAN/images/'+picture)
tmp = cv2.imread('/content/GFPGAN/results/restored_imgs/'+picture)
color = cv2.imread('/content/DeOldify/result_images/'+picture)
height, width ,channels =tmp.shape
mono = cv2.resize(mono, dsize=(width, height))
color = cv2.resize(color, dsize=(width, height))

if height > width:
  result = cv2.hconcat([mono, color])
else:
  result = cv2.vconcat([color, mono])  
cv2.imwrite('compare_images/fine'+picture, result)

# display
from IPython.display import display
from IPython.display import Image as show_img
display(show_img('compare_images/fine'+picture))

# auto-download
from google.colab import files
files.download('compare_images/fine'+picture)